# Einführung in R Data Frames

### Vorwort
Dies ist eine Übertragung eines Teils des Tutorials von [Greg Reda](http://www.gregreda.com/2013/10/26/intro-to-pandas-data-structures/) für Pandas, der Python-Bibliothek für Data Frames, auf R.

Informationen über die meisten der hier verwendeten Methoden findet man in den [Manuals auf CRAN](https://cran.r-project.org/manuals.html).

## Data Frame

In R ist ein DataFrame ein grundlegendes Objekt, welches ohne zusätzliche Pakete zur Verfügung steht. Dem Im- und Export von Daten widmet sich ein eigenes [Handbuch](https://cran.r-project.org/doc/manuals/r-release/R-data.html).

### Anlegen eines Data Frames

Man kann ein Data Frame beispielsweise wie folgt von Hand anlegen:

In [ ]:
football <- data.frame(year = c(2010, 2011, 2012, 2011, 2012, 2010, 2011, 2012),
        team = c('Bears', 'Bears', 'Bears', 'Packers', 'Packers', 'Lions', 'Lions', 'Lions'),
        wins = c(11, 8, 10, 15, 11, 6, 10, 4),
        losses = c(5, 8, 6, 1, 5, 10, 6, 12))
football

Datensätze werden allerdings nur selten wie eben geschehen per Hand angelegt, sondern eingelesen, sei es aus Dateien oder über die Verbindung zu einer Datenbank. Dies wird im Folgenden dargestellt.

**CSV**

Eine CSV-Datei kann  man einfach mit Hilfe der Funktion *read.csv()* einlesen. Dabei wird das *Komma* als Feldtrenner vorausgesetzt; dagegen erwartet *read.csv2()* ein *Semikolon* und außerdem wird hier ein *Komma als Dezimaltrenner* interpretiert! Will man flexibel den Feldtrenner mit Hilfe des `sep`-Parameters selbst setzen, so verwendet man die allgemeine Funktion *read.table()*, die auch noch andere Einstellungen wie etwa das *Encoding* erlaubt (siehe [Spreadsheet-like Data](https://cran.r-project.org/doc/manuals/r-release/R-data.html#Spreadsheet_002dlike-data))

In [ ]:
from_csv <- read.csv('data/mariano-rivera.csv', stringsAsFactors = FALSE)
head(from_csv)

Zuvor wurde die erste Zeile als Kopfzeile mit den Spaltennamen interpretiert. Haben wir eine reine Datendatei, dann müssen wir *read.table()* verwenden mit der Angabe `header=FALSE`. Die Spaltennamen können dann auch mit Hilfe des Parameters `col.names` spezifiziert werden: 

In [ ]:
cols <- c('num', 'game', 'date', 'team', 'home_away', 'opponent',
        'result', 'quarter', 'distance', 'receiver', 'score_before',
        'score_after')
no_headers <- read.table('data/peyton-passing-TDs-2012.csv', sep=',', header=FALSE,
                         stringsAsFactors = FALSE, col.names=cols)
head(no_headers)

Ein Abschnitt des Handbuchs befasst sich speziell mit dem [Export in Textdateien](https://cran.r-project.org/doc/manuals/r-release/R-data.html#Export-to-text-files).

**Excel**

Für den Umgang mit Excel-Dateien benötigt man eine eigene Bibliothek. Es gibt dafür verschiedene Möglichkeiten. Hier wird [openxlsx](https://www.rdocumentation.org/packages/openxlsxversions/4.0.0) verwendet, da das Paket ohne Java-Abhängigkeiten auskommt. Allerdings muss es zunächst installiert werden (siehe die Beschreibung im oben angegebenen Link).

In diesem Zusammenhang ist auch von Interesse, was die [R-Doku](https://cran.r-project.org/doc/manuals/r-release/R-data.html#Reading-Excel-spreadsheets) dazu meint ...

In [ ]:
# Paket laden
library(openxlsx)

In [ ]:
# mal schauen, was es kann
help(openxlsx)

Wir schreiben das zuvor händisch angelegte Data Frame in eine Excel-Datei.

In [ ]:
# tail gibt es in R auch
tail(football)

In [ ]:
# in eine Excel-Datei schreiben
write.xlsx(football,'data/football-R.xlsx')

In [ ]:
# prüfen, ob die Datei angelegt wurde
system2('ls', args='-l data/*.xlsx', stdout=TRUE)

In [ ]:
# Data Frame löschen
rm(football)

In [ ]:
# Excel einlesen
football <- read.xlsx('data/football-R.xlsx')
football

**Datenbanken**

Zur Verbindung mit Datenbanken ist es sinnvoll, das Paket [DBI](https://www.rdocumentation.org/packages/DBI) zu verwenden. Ferner benötigt man noch spezielle Pakete für die einzelnen DBMS, mit denen man arbeiten möchte; in unserem Fall [RSQLite](https://www.rdocumentation.org/packages/RSQLite). Zu dieser Kombination existiert auch eine [Vignette](https://cran.r-project.org/web/packages/RSQLite/vignettes/RSQLite.html).

In [ ]:
library("DBI")
# library("RSQLite")

conn <- dbConnect(RSQLite::SQLite(), 'data/towed.db')
query <- "SELECT * FROM towed WHERE make = 'FORD';"

results <- dbGetQuery(conn, query)
head(results)

**Zwischenablage (Clipboard)**

Auch das Einlesen aus der Zwischenablage ist einfach möglich. Nach KLick auf den folgenden Link markiere man die Daten wie im Bild zu sehen und führe dann die nachfolgende Zelle aus.

[Hank Aaron](http://www.baseball-reference.com/players/a/aaronha01.shtml)

![hank-aaron-stats-screenshot](http://up.picr.de/29101294uq.jpg)

In [ ]:
hank <- read.delim("clipboard", stringsAsFactors = FALSE)
head(hank)

**URL**

Mittels `read.table()` kann man auch direkt von einer URL lesen. Wir benutzen wieder Greg Redas [best sandwiches Daten](https://raw.github.com/gjreda/best-sandwiches/master/data/best-sandwiches-geocode.tsv).

In [ ]:
url = 'https://raw.github.com/gjreda/best-sandwiches/master/data/best-sandwiches-geocode.tsv'

# fetch the text from the URL and read it into a DataFrame
from_url = read.table(url, header=TRUE, sep='\t', stringsAsFactors = FALSE)
head(from_url,3)

## Mit Data Frames arbeiten

Wir benutzen auch hier den [MovieLens](http://www.grouplens.org/node/73) Datensatz wie bei der Pandas-Einführung.

Im Folgenden werden zunächst SQL-Statements gezeigt, normalerweise für MySQL, aber manchmal auch für SQLite. Anschließend folgen die entsprechenden Python-Äquivalente mit Pandas.

```SQL
CREATE TABLE users (
  user_id int,
  age smallint,
  sex varchar(200),
  occupation varchar(200),
  zip_code varchar(200)
);

LOAD DATA LOCAL INFILE 'ml-100k/u.user'
INTO TABLE users
CHARACTER SET 'latin1'
FIELDS TERMINATED BY '|'
LINES TERMINATED BY '\n';
```
In R wird also der Zeichensatz genauso bezeichnet wie in MySQL (`'latin1'`), während man in Python `'latin-1'` verwenden muss!

In [ ]:
# Spaltennamen separat festlegen
u_cols <- c('user_id', 'age', 'sex', 'occupation', 'zip_code')
users <- read.table('data/ml-100k/u.user', sep='|', col.names=u_cols, header=FALSE,
                    fileEncoding='latin1')
head(users)

```SQL
CREATE TABLE ratings (
  user_id int,
  movie_id int,
  rating tinyint,
  unix_timestamp bigint
);

LOAD DATA LOCAL INFILE 'ml-100k/u.data'
INTO TABLE ratings
CHARACTER SET 'latin1'
FIELDS TERMINATED BY '\t'
LINES TERMINATED BY '\n';
```
> Hier ist der ***Spaltentrenner*** also ein Tabulator, kein Pipe-Zeichen!

In [ ]:
r_cols <- c('user_id', 'movie_id', 'rating', 'unix_timestamp')
ratings = read.csv('data/ml-100k/u.data', sep='\t', col.names=r_cols, header=FALSE,
                      fileEncoding='latin1')
head(ratings,3)

```SQL
CREATE TABLE movies (
  movie_id int,
  title varchar(200),
  release_date varchar(200),
  video_release_date varchar(200),
  imdb_url varchar(200)
);

-- unused columns at the end automatically ignored
LOAD DATA LOCAL INFILE 'ml-100k/u.item'
INTO TABLE movies
CHARACTER SET 'latin1'
FIELDS TERMINATED BY '|'
LINES TERMINATED BY '\n';
```

Das Einlesen eines Datensatzes mit mehr Spalten als man braucht ist etwas aufwändiger. Im folgenden Fall benötigen wir nur die ersten 5 Spalten.

In [ ]:
# Spaltennamen
m_cols <- c('movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url')
# erst einmal alles einlesen
movies <- read.csv('data/ml-100k/u.item', sep='|', header=FALSE,
                     fileEncoding='latin1')
# nur die ersten fünf Spalten behalten
movies <- movies[1:5]
# Namen festlegen
colnames(movies) <- m_cols
head(movies,3)

### Beschreibung

Meta-Information zum Datensatz erhält man mittels `str` bzw. `summary.default()`.

> #### SQL-Äquivalent:
für MySQL:
```SQL
DESCRIBE towed;
```
für SQLite:
```SQL
PRAGMA table_info([towed]);
```

In [ ]:
str(movies)

In [ ]:
summary.default(movies)

Wir haben bereits mehrfach die `head`-Methode verwendet. In R werden dabei per default die ersten 6 Zeilen ausgegeben, mit `tail` die letzten 6.

> #### SQL-Äquivalent
 zu `head()`
```SQL
SELECT * 
FROM movies 
LIMIT 6;
```

In [ ]:
head(movies)

> #### SQL-Äquivalent
zu `tail(,3)`<br/>
Dies funktioniert nur in SQLite wegen der Pseudo-Spalte `ROWID`!
```SQL
SELECT * 
FROM movies 
ORDER BY ROWID DESC
LIMIT 3;
```

In [ ]:
tail(movies,3)

Man kann auch bestimmte Zeilen per [Referenzierung](https://de.wikibooks.org/wiki/GNU_R:_Umgang_mit_Datens%C3%A4tzen_(Erstellen,_Ausw%C3%A4hlen_und_Filtern)) auswählen!

> #### SQL-Äquivalent
zu `[21:22,]` <br/>
Dies funktioniert nur in SQLite wegen der Pseudo-Spalte `ROWID`. Wi in SQLite beginnt auch in R der Index bei 1!
```SQL
SELECT * 
FROM movies 
WHERE ROWID IN (21,22);
```

In [ ]:
movies[21:22,]

### Selektion

Auswählen kann man einfach per Angabe des Spaltennamens.

> #### SQL-Äquivalent
```SQL
SELECT occupation
FROM users
LIMIT 5;
```

In [ ]:
head(users['occupation'])

Mehrere Spalten müssen erst wieder kombiniert werden.

> #### SQL-Äquivalent

```SQL
SELECT age, zip_code
FROM users
LIMIT 5;

SELECT occupation, sex
FROM users
LIMIT 5;
```

In [ ]:
head(users[c('age', 'zip_code')])
# can also store in a variable to use later
columns_you_want <- c('occupation', 'sex')
print(head(users[columns_you_want]))

#### Zeilenauswahl
Dies kann mit einem Index oder einem logischen Ausdruck geschehen. Die fragliche Spalte kann man wie gewohnt angeben, aber auch mit dem Dollarzeichen $\$$.

> #### SQL-Äquivalent (WHERE Klausel)

```SQL
SELECT *
FROM users
WHERE age > 25
LIMIT 3;

SELECT *
FROM users
WHERE age = 40 AND sex = 'M'
LIMIT 3;

SELECT *
FROM users
WHERE sex = 'F' OR age < 30
LIMIT 3;
```

In [ ]:
# users older than 25
print(head(subset(users, users$age > 25),3))
cat("\n")
# users aged 40 AND male
print(head(subset(users, users['age'] == 40 & users['sex'] == 'M'),3))
cat("\n")
# users younger than 30 OR female
print(head(subset(users, users$sex == 'F' | users$age < 30),3))

Der Index für unsere Zeilen im `users` Data Frame ist lediglich quasi eine Kopie der Spalte `user_id`. In der R-Terminologie ist dies der *Zeilenname*. Um also Zeilen über den Index zu referenzieren, wie wir es mit Primärschlüsseln aus relationalen Datenbanken gewohnt sind, müssen wir dies spezifizieren.

In [ ]:
rownames(users) <- users$user_id
head(users,3)

### Maskierung
Man kann *Filtermasken* definieren und damit einfach durch Bool'sche Indizierung arbeiten. In SQL geht das prinzipiell auch, ist aber recht unpraktisch ... Wir zeigen das an den weiblichen Benutzern unter 40, zunächst mit dem
#### SQL Äquivalent
```SQL
CREATE VIEW females_under_40 AS
SELECT *,
CASE 
WHEN age < 40 AND sex = 'F' THEN 1
ELSE 0
END AS mask 
FROM users;
```
und dann auch in R:

In [ ]:
fu40 <- (users$age < 40 & users['sex'] == 'F')

Nun wüssten wir gerne, wieviele Frauen unter 40 wir denn haben
#### SQL-Äquivalent
```SQL
SELECT mask, COUNT(*) 
FROM females_under_40
GROUP BY mask;
```

In [ ]:
summary.default(fu40)

Und schleßlich noch die ersten 6 der Damen unter 40 (das Geschlecht brauchen wir nicht ...)
### SQL-Äquivalent
```SQL
SELECT user_id, age, occupation, zip_code
FROM females_under_40
WHERE mask = 1
LIMIT 6;
```

In [ ]:
head(subset(users[c('age','occupation','zip_code')],fu40)) # user_id ist ja unser Index und wird automatisch angezeigt

### Index basierte Auswahl (Referenzierung)
Wir können einzelne Zeilen auswählen (siehe auch <a href="https://de.wikibooks.org/wiki/GNU_R:_Umgang_mit_Datens%C3%A4tzen_(Erstellen,_Ausw%C3%A4hlen_und_Filtern)#Referenzierung_.28Auswahl_von_Variablen_oder_F.C3.A4llen.29">hier</a>)

> #### SQL-Äquivalent
Dies funktioniert nur in SQLite wegen der Pseudo-Spalte `ROWID`!

```SQL
SELECT *
FROM users
WHERE ROWID = 100;

SELECT *
FROM users
WHERE ROWID IN (2, 51, 301);
```

In [ ]:
print(users[100,])
cat('\n')
print(users[c(2, 51, 301),])

### Joining

Die Verbindung von Datensätzen ist bei Analysen an der Tagesordnung. In R steht dafür die Funktion `merge` zur Verfügung. Eine Übersicht über die Möglichekietn erhält man bei [Stackoverflow](http://stackoverflow.com/questions/1299871/how-to-join-merge-data-frames-inner-outer-left-right) oder in Verbindung mit dem [R Cookbook](http://www.cookbook-r.com/Manipulating_data/Merging_data_frames/) oder im [Handbuch](https://stat.ethz.ch/R-manual/R-devel/library/base/html/merge.html).

In [ ]:
left_frame <- data.frame(key = c(1:5), 
                           left_value = c('a', 'b', 'c', 'd', 'e'))
right_frame = data.frame(key = c(2:6), 
                           right_value = c('f', 'g', 'h', 'i', 'j'))
print(left_frame)
cat('\n')
print(right_frame)

**inner join (default)**

#### SQL Äquivalent:

```SQL
    SELECT left_frame.key, left_frame.left_value, right_frame.right_value
    FROM left_frame
    INNER JOIN right_frame
        ON left_frame.key = right_frame.key;
```

In [ ]:
merge(x=left_frame, y=right_frame)

Hätte unsere Join-Spalte nicht in beiden Fällen *key* gehießen, so hätten wir die zu verbindenden Spalten durch *by.x* und *by.y* spezifizieren können:
```r
    merge(x=left_frame, y=right_frame, by.x='left_key', by.y='right_key')
```

**left outer join**

#### SQL Äquivalent:
```SQL
    SELECT left_frame.key, left_frame.left_value, right_frame.right_value
    FROM left_frame
    LEFT JOIN right_frame
        ON left_frame.key = right_frame.key;
```

In [ ]:
merge(left_frame, right_frame, by='key', all.x=TRUE)

Wie man sieht, kann man auch die Namen der Parameter in der Angabe weglassen.

**right outer join**

#### SQL Äquivalent:
```SQL
    SELECT right_frame.key, left_frame.left_value, right_frame.right_value
    FROM left_frame
    RIGHT JOIN right_frame
        ON left_frame.key = right_frame.key;
```

In [ ]:
merge(left_frame, right_frame, by='key', all.y=TRUE)

**full outer join**

#### SQL Äquivalent 
(soweit möglich (z.B. nicht in MySQL)):
```SQL
    SELECT IFNULL(left_frame.key, right_frame.key) key
            , left_frame.left_value, right_frame.right_value
    FROM left_frame
    FULL OUTER JOIN right_frame
        ON left_frame.key = right_frame.key;
```

In [ ]:
merge(left_frame, right_frame, by='key', all=TRUE)

Bisher haben wir zur Datenmanipulation nur eingebaute Funktionen von R benutzt (nur für den Umgang mit Dateien und Datenbenaken hatten wir spezielle Pakete verwendet). Allerdings gibt es auch ein spezielles Paket, welches den Umgang mit Data Frames nochmal deutlich erleichtert, nämlich [dplyr](https://www.rdocumentation.org/packages/dplyr). Es wäre eine gute **Übung**, die bisherigen Datenmanipulationen mit Hilfe dieses Pakets zu wiederholen. Für die folgende Gruppierung werden wir es in jedem Fall verwenden.

### Gruppierung
Die bekannten Aggregatfunktionen sind im `dplyr`-Paket einfach realisiert und auch wieder ähnlich zu Pandas. Wie dort auch gibt es Aggregatfunktionen wie `median`, die in SQL nicht zur Verfügung stehen.

In [ ]:
library(dplyr)

Wir benutzen wieder die Daten der Einkommen der Bediensteten der Stadt Chicago. Allerdings ist diesmal das Dollarzeichen mit dem Editor entfernt worden ...

In [ ]:
system2('head',args='-n 3 data/city-of-chicago-salaries_prep.csv',stdout=TRUE)

In [ ]:
headers <- c('name', 'title', 'department', 'salary')
chicago <- read.csv('data/city-of-chicago-salaries_prep.csv', 
                      header=TRUE, stringsAsFactors = FALSE,
                      col.names=headers)
head(chicago)

Mittels `group_by` kann man die aggregierten Auswertungen vorbereiten, wobei man zunächst in der Ausgabe keinen Unterschied sieht zum ursprünglichen Data Frame. Der Unterschied wird aber klar, wenn man die Strukturen vergleicht.
>**Anmerkung** Wer mehr über Syntax wie `%>%` erfahren möchte, dem sei [dieser Blogeintrag](https://martinsbioblogg.wordpress.com/2014/03/27/more-fun-with-and/) ans Herz gelegt ...

In [ ]:
by_dept <- chicago %>% group_by(department)
head(by_dept)

In [ ]:
str(chicago)

In [ ]:
str(by_dept)

### Aggregate
Die Bildung von Aggregaten geschieht mittels `summarise`. Die folgenden Beispiele sollten (zumindest in Kombination mit den SQL-Äquivalenten) weitgehend selbsterklärend sein (wenn man sich an die Notation gewöhnt hat ...).

> #### SQL-Äquivalent

```SQL
SELECT department, COUNT(name) AS name, COUNT(title) AS title, COUNT(salary) AS salary
FROM chicago
GROUP BY department
LIMIT 5;

SELECT department, COUNT(*)
FROM chicago
GROUP BY department
ORDER BY department DESC
LIMIT 5;
```

In [ ]:
head(by_dept %>% summarise(name = length(name), title=length(title), salary=length(salary)))
tail(by_dept %>% summarise(n = length(name)))

> #### SQL-Äquivalent

```SQL
-- sum, mean, (no) median ;-(
SELECT department, SUM(salary) AS total_salary, AVG(salary) AS avg, 'N/A' AS med
FROM chicago
GROUP BY department
LIMIT 6;
```

In [ ]:
head(by_dept %>% summarise(total_salary=sum(salary), avg=mean(salary), med=median(salary)))

Nun suchen wir die 5 Abteilungen mit dem meisten unterschiedlichen Jobtiteln.

```SQL
    SELECT department, COUNT(DISTINCT title)
    FROM chicago
    GROUP BY department
    ORDER BY 2 DESC
    LIMIT 5;
```

In [ ]:
head(by_dept %>% summarise(n = n_distinct(title)) %>% arrange(desc(n)),5) 